In [1]:
from xgboost import XGBRegressor
import pandas as pd
from functions import pathfile

In [3]:
pf=pathfile()
df=pd.read_csv(pf)
df.head()


C:\Users\ingva\OneDrive\Documents\EstudioML\data\raw\melb_data.csv


,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [28]:
cols_num=[cname for cname in df.columns if df[cname].dtypes in ['int64','float64']]
print('Col_num: ',cols_num)
cols_cat=[cname for cname in df.columns if df[cname].dtypes=='object' and len(df[cname].unique())<=10]
print('Col_num: ',cols_cat)

Col_num:  ['Rooms', 'Price', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']
Col_num:  ['Type', 'Method', 'Regionname']


In [33]:
#Imputamos los valores nulls en var num
cols_num_nulls=[]

for col in cols_num:
   if df[col].isnull().any()==True:
      cols_num_nulls.append(col)
      df[col+'_missing']=df[col].isnull()
      df[col]=df[col].fillna(df[col].mean())
df.head()

#las variables seleccionadas son con cardinal menor igual 10
#Revisaremos la cantidad de nulos en estas catergorias
#si no tenemos nulos realizaremos el one hot encode

col_cat_null={}

for col in cols_cat:
   if df[col].isnull().any()==True:
      col_cat_null[col]=df[col].isnull().value_counts()
print(col_cat_null)


{}


In [35]:
df[cols_cat].isnull().any()

Type          False
Method        False
Regionname    False
dtype: bool

In [58]:
#Haremos OneHotEncode
df_dummies_cat=pd.get_dummies(df[cols_cat],columns=cols_cat,dtype='int')
df_dummies_cat.head().reset_index()

,index,Type_h,Type_t,Type_u,Method_PI,Method_S,Method_SA,Method_SP,Method_VB,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria
0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
2,2,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0
3,3,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4,4,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0


In [61]:
#unificamos en un solo df
df_f=pd.concat([df[cols_num],df_dummies_cat],axis=1)
df_f.head()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,...,Method_SP,Method_VB,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria
0,2,1480000.0,2.5,3067.0,2.0,1.0,1.0,202.0,151.96765,1964.684217,...,0,0,0,0,1,0,0,0,0,0
1,2,1035000.0,2.5,3067.0,2.0,1.0,0.0,156.0,79.00000,1900.000000,...,0,0,0,0,1,0,0,0,0,0
2,3,1465000.0,2.5,3067.0,3.0,2.0,0.0,134.0,150.00000,1900.000000,...,1,0,0,0,1,0,0,0,0,0
3,3,850000.0,2.5,3067.0,3.0,2.0,1.0,94.0,151.96765,1964.684217,...,0,0,0,0,1,0,0,0,0,0
4,4,1600000.0,2.5,3067.0,3.0,1.0,2.0,120.0,142.00000,2014.000000,...,0,1,0,0,1,0,0,0,0,0


In [62]:
from sklearn.model_selection import train_test_split
y=df_f.Price
X=df_f.drop('Price',axis=1)
print(X.columns)

Index(['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount', 'Type_h', 'Type_t', 'Type_u', 'Method_PI', 'Method_S',
       'Method_SA', 'Method_SP', 'Method_VB',
       'Regionname_Eastern Metropolitan', 'Regionname_Eastern Victoria',
       'Regionname_Northern Metropolitan', 'Regionname_Northern Victoria',
       'Regionname_South-Eastern Metropolitan',
       'Regionname_Southern Metropolitan', 'Regionname_Western Metropolitan',
       'Regionname_Western Victoria'],
      dtype='object')


In [63]:
X.head()


,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,...,Method_SP,Method_VB,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria
0,2,2.5,3067.0,2.0,1.0,1.0,202.0,151.96765,1964.684217,-37.7996,...,0,0,0,0,1,0,0,0,0,0
1,2,2.5,3067.0,2.0,1.0,0.0,156.0,79.00000,1900.000000,-37.8079,...,0,0,0,0,1,0,0,0,0,0
2,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.00000,1900.000000,-37.8093,...,1,0,0,0,1,0,0,0,0,0
3,3,2.5,3067.0,3.0,2.0,1.0,94.0,151.96765,1964.684217,-37.7969,...,0,0,0,0,1,0,0,0,0,0
4,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.00000,2014.000000,-37.8072,...,0,1,0,0,1,0,0,0,0,0


In [80]:
from sklearn.metrics import mean_absolute_error
X_train,X_valid,y_train,y_valid=train_test_split(X,y,random_state=0,train_size=0.7,test_size=0.3)
model=XGBRegressor(n_estimators=1000,learning_rate=0.05,n_jobs=4)
model.fit(X_train,y_train,early_stopping_rounds=5,eval_set=[(X_valid,y_valid)],verbose=False)
print("score: ",model.score(X_train,y_train))
predicts=model.predict(X_valid)
mae=mean_absolute_error(predicts,y_valid)
print("mae= ",mae)

c:\Users\ingva\anaconda3\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


score:  0.9296887273696316
mae=  157561.94714577196


In [75]:
#aplicamos un cross validation
from sklearn.model_selection import cross_val_score

score=-1*cross_val_score(model,X,y,cv=5,scoring='neg_mean_absolute_error',n_jobs=4)
print("mae_scores: ",score)

mae_scores:  [197684.70651348 180175.32853346 172796.15024968 149092.15949466
 151733.96986032]


In [78]:
cross_score=score.mean()
cross_score


170296.46293032033

In [85]:
percent=((cross_score-mae)/mae)*100
percent

8.082227984125343